### Boosting Decisions

En este cuaderno, aprenderemos del proceso de "boosting" usando Boosting Adaptivo y los métodos de Árboles 'Boosted' con Gradiente. (Nota: "boost" aquí significa alzar o dar impulso, en el sentido de aumentar o empoderar algo.)

La aplicación final será para el problema de corrimientos al rojo fotométricos del Capítulo 6; la solución particular para esos se verá en el cuaderno "Tipos de Boosting."

Autor: Viviana Acquaviva, con contibuciones de Jake Postiglione y Olga Privman. Traducido por Lucia Perez y Rosario Cecilio-Flores-Elie. 

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)


font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
#matplotlib.rcParams.update({'figure.autolayout': True})
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_validate, KFold, cross_val_predict, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, GradientBoostingRegressor

Referencia que compara diferentes aprendices débiles cuando actúan como estimadores de base:

https://link.springer.com/chapter/10.1007/978-3-642-20042-7_32

Ejemplos de implementaciones empezando de casi nada (usando pesas de muestra):

https://xavierbourretsicotte.github.io/AdaBoost.html


https://geoffruddock.com/adaboost-from-scratch-in-python/

### Primero imputamos en conjunto de los corrimientos al rojo fotométricos (ya con selecciones aplicadas) que creamos en el último cuaderno.

In [ ]:
sel_features = pd.read_csv('../data/sel_features.csv', sep = '\t')

In [ ]:
sel_target = pd.read_csv('../data/sel_target.csv')

In [ ]:
sel_features.shape

In [ ]:
sel_target.values.ravel() #cambia de forma a una matriz en forma de fila 1d

### Podemos probar nuestro proceso de referencia con AdaBoost, usando los valores por defecto.

In [ ]:
model = AdaBoostRegressor()

In [ ]:
ypred = cross_val_predict(model, sel_features,sel_target.values.ravel(), cv = KFold(n_splits=5, shuffle=True, random_state=10))

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(sel_target,ypred, s =10)
plt.ylim(0,3)
plt.xlim(0,3)

### ¿Te parece que el proceso de !!!boosting está funcionando? A mi no.

In [ ]:
model.get_params()

Nota (de la documentacion de sklearn): Si "None", el estimador de base es "DecisionTreeRegressor(max_depth=3)".

### Decidí investigar como diferentes parámetros afectan el desempeño.

#### Cambiar la profundidad máxima de los estimadores de base: intente usar árboles con no mas de 3, 6, o 10 partidas.

In [ ]:
plt.figure(figsize=(12,4))

for i, depth in enumerate([3,6,10]):
    plt.subplot(1,3,i+1)
    model = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=depth))
    ypred = cross_val_predict(model, sel_features,sel_target.values.ravel(), cv = KFold(n_splits=5, shuffle=True, random_state=10))
    plt.scatter(sel_target,ypred, s =10, c = 'teal')
    plt.title('Max depth = '+str(depth))
    plt.xlabel('True redshift')
    if i == 0:
        plt.ylabel('Estimated redshift')
    plt.ylim(0,2)
    plt.xlim(0,2)
    
    plt.tight_layout()

#plt.savefig('AdaB_z.png')
#    plt.axes('equal')
#    plt.legend()

#### Cambiar en Número de estimadores de base (o, el número de etapas en el !!!boosting).

In [ ]:
plt.figure(figsize=(7,7))

plt.ylim(0,2)
plt.xlim(0,2)

for nest in [5,10,20]:
    model = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=6), n_estimators=nest)
    ypred = cross_val_predict(model, sel_features,sel_target.values.ravel(), cv = KFold(n_splits=5, shuffle=True, random_state=10))
    plt.scatter(sel_target,ypred, s =10, label = 'N est = '+str(nest))
plt.legend()

#### Cambiar la función de pérdida

In [ ]:
plt.figure(figsize=(7,7))

plt.ylim(0,2)
plt.xlim(0,2)

for loss in ['linear','square']:
    model = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=6), loss = loss)
    ypred = cross_val_predict(model, sel_features,sel_target.values.ravel(), cv = KFold(n_splits=5, shuffle=True, random_state=10))
    plt.scatter(sel_target,ypred, s =10, label = 'Loss = '+loss)
plt.legend();

### ¿Qué aprendimos? Para AdaBoost, el estimador de base necesita ser suficientemente 'fuerte' para tener éxito en !!!boosting .

## Ejemplo básico de regresión
### Inspirado por...

https://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_regression.html#sphx-glr-auto-examples-ensemble-plot-adaboost-regression-py

#### ¿Qué pasa si max_depth = 3?

In [ ]:
# Creamos el conjunto de datos
plt.figure(figsize=(15,10))

rng = np.random.RandomState(1)
X = np.linspace(0, 4, 100)[:, np.newaxis]
y = np.sin(X).ravel() + np.sin(6 * X).ravel() + rng.normal(0, 0.1, X.shape[0])

weakl = DecisionTreeRegressor(max_depth=3)

# Encajamos el modelo de regresión, y guardamos cada 'etapa'

regr_1 = weakl
""
regr_2 = AdaBoostRegressor(weakl,
                          n_estimators=2, random_state=rng)

regr_3 = AdaBoostRegressor(weakl,
                          n_estimators=3, random_state=rng)

regr_4 = AdaBoostRegressor(weakl,
                          n_estimators=4, random_state=rng)

regr_10 = AdaBoostRegressor(weakl,
                          n_estimators=10, random_state=rng)

regr_100 = AdaBoostRegressor(weakl,
                          n_estimators=100, random_state=rng)


regr_1.fit(X, y)
regr_2.fit(X, y)
regr_3.fit(X, y)
regr_4.fit(X, y)
regr_10.fit(X, y)
regr_100.fit(X, y)

# Predicción
y_1 = regr_1.predict(X)
y_2 = regr_2.predict(X)
y_3 = regr_3.predict(X)
y_4 = regr_4.predict(X)
y_10 = regr_10.predict(X)

for yp in [y_1,y_2,y_3,y_4,y_10]:
    print('r2 score: ', np.round(metrics.r2_score(yp,y),3))

# Marcamos los resultados 

plt.scatter(X, y, c="k", s=10,label="training samples")
plt.plot(X, y_1, "-g", label="n_estimators=1", linewidth=1)
plt.plot(X, y_2, "--r", label="n_estimators=2", linewidth=1)
plt.plot(X, y_3, "-.b", label="n_estimators=3", linewidth=1)
#plt.plot(X, y_4, ":m", label="n_estimators=4", linewidth=1)
#plt.plot(X, y_10, "-k", label="n_estimators=10", linewidth=1)
plt.xlabel("data")
plt.ylabel("target")
plt.title("AdaBoost Regression, max depth = 3", fontsize = 14)
plt.legend(fontsize=10);
#plt.tight_layout()
#plt.savefig("AdaBoost_3.png")

#### ¿Qué pasa si max_depth = 6?

In [ ]:
# Create the dataset
plt.figure(figsize=(15,10))

rng = np.random.RandomState(1)
X = np.linspace(0, 4, 100)[:, np.newaxis]
y = np.sin(X).ravel() + np.sin(6 * X).ravel() + rng.normal(0, 0.1, X.shape[0])

weakl = DecisionTreeRegressor(max_depth=6)

# Encajamos el modelo de regresión, y guardamos cada 'etapa'
regr_1 = weakl
""
regr_2 = AdaBoostRegressor(weakl,
                          n_estimators=2, random_state=rng)

regr_3 = AdaBoostRegressor(weakl,
                          n_estimators=3, random_state=rng)

regr_4 = AdaBoostRegressor(weakl,
                          n_estimators=4, random_state=rng)

regr_10 = AdaBoostRegressor(weakl,
                          n_estimators=10, random_state=rng)

regr_100 = AdaBoostRegressor(weakl,
                          n_estimators=100, random_state=rng)


regr_1.fit(X, y)
regr_2.fit(X, y)
regr_3.fit(X, y)
regr_4.fit(X, y)
regr_10.fit(X, y)
regr_100.fit(X, y)

# Predicción
y_1 = regr_1.predict(X)
y_2 = regr_2.predict(X)
y_3 = regr_3.predict(X)
y_4 = regr_4.predict(X)
y_10 = regr_10.predict(X)

for yp in [y_1,y_2,y_3,y_4,y_10]:
    print(metrics.r2_score(yp,y))

# Marcamos los resultados 

plt.scatter(X, y, c="k", s=10,label="training samples")
plt.plot(X, y_1, "-g", label="n_estimators=1", linewidth=1)
plt.plot(X, y_2, "--r", label="n_estimators=2", linewidth=1)
plt.plot(X, y_3, "-.b", label="n_estimators=3", linewidth=1)
#plt.plot(X, y_4, ":m", label="n_estimators=4", linewidth=1)
#plt.plot(X, y_10, "-k", label="n_estimators=10", linewidth=1)
plt.xlabel("data")
plt.ylabel("target")
plt.title("AdaBoost Regression, max depth = 6", fontsize = 14)
plt.legend(fontsize=10);
#plt.tight_layout()
#plt.savefig("AdaBoost_6.png")
plt.show()

### Revisión de aprendizaje

Mirando a la figura que creamos, ¿vale la pena hacer el proceso de boosting con AdaBoost si el aprendiz de base tiene max_depth = 3? ¿Vale la pena si max_depth = 6?

<br>

<details>
<summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
<p>
    
```
No vale la pena para max_depth = 3--las notas de r2 no se mejoran si usamos mas estimadores. Tal vez valga la pena para max_depth = 6, pero las notas son estables, y por eso  tal vez no se necesita mas investigación. 
```
    
</p>
</details>

### Ahora que estamos convencidos, vamos a volver a los photo-zs.

Primero se crea una partida entre entrenamiento y prueba, para poder accesar la propiedad "staged_predict".

In [ ]:
X_train, X_test, y_train, y_test = \
        train_test_split(sel_features,sel_target.values.ravel(), test_size=.3, random_state=42)

In [ ]:
# empezamos con un aprendiz muy débil (r2 = 0.4) 

model= AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=3),
                  n_estimators=30)

In [ ]:
model.fit(X_train, y_train)

Podemos marcar la nota de r2 y el coeficiente de correlación Spearman entre los valores verdaderos y previstos, como una función del número de etapas/iteración, empezando con un aprendiz muy débil. 

In [ ]:
n_estimators = 30

plt.plot(range(n_estimators), [metrics.r2_score(y_test,list(model.staged_predict(X_test))[i]) for i in range(n_estimators)], label = 'r2 score')

plt.plot(range(n_estimators), [stats.spearmanr(y_test,list(model.staged_predict(X_test))[i])[0] for i in range(n_estimators)], label = 'Spearman r')

plt.xlabel('Iteration')

plt.ylim(0,1.0)

plt.title('Max depth = 3')
plt.legend();

### Las notas no se mejoran con más estimadores.

Intentemoslo de nuevo con un aprendiz de base más fuerte (max_depth = 6). 

In [ ]:
n_estimators = 30

model= AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=6),
                  n_estimators=n_estimators)

X_train, X_test, y_train, y_test = \
        train_test_split(sel_features,sel_target.values.ravel(), test_size=.3, random_state=42)

model.fit(X_train, y_train)

plt.plot(range(n_estimators), [metrics.r2_score(y_test,list(model.staged_predict(X_test))[i]) for i in range(n_estimators)], label = 'r2')

plt.plot(range(n_estimators), [stats.spearmanr(y_test,list(model.staged_predict(X_test))[i])[0] for i in range(n_estimators)], label = 'Spearman r')

plt.xlabel('Iteration')

plt.title('Base estimator, max depth = 6')
plt.legend();

... y un aprendiz de base aún más fuerte (max_dpeth = 10).

In [ ]:
n_estimators = 30

model= AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=10),
                  n_estimators=30)

X_train, X_test, y_train, y_test = \
        train_test_split(sel_features,sel_target.values.ravel(), test_size=.3, random_state=42)

model.fit(X_train, y_train)

plt.plot(range(n_estimators), [metrics.r2_score(y_test,list(model.staged_predict(X_test))[i]) for i in range(n_estimators)], label = 'r2')

plt.plot(range(n_estimators), [stats.spearmanr(y_test,list(model.staged_predict(X_test))[i])[0] for i in range(n_estimators)], label = 'Spearman r')

plt.xlabel('Iteration')

plt.title('Base estimator, max depth = 10')

plt.legend();

### Miremoslo todos en una figura.

In [ ]:
plt.figure(figsize=(12,4))

n_estimators = 30

for i, md in enumerate([3,6,10]):
    
    model = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=md),
                  n_estimators=n_estimators)

    model.fit(X_train,y_train)
    
    plt.subplot(1,3,i+1)

    plt.plot(range(n_estimators), [metrics.r2_score(y_test,list(model.staged_predict(X_test))[i]) for i in range(n_estimators)], label = 'r2 score', c = 'steelblue')

    plt.plot(range(n_estimators), [stats.spearmanr(y_test,list(model.staged_predict(X_test))[i])[0] for i in range(n_estimators)], label = 'Spearman r', c = 'fuchsia')

    plt.xlabel('Iteration')

    plt.ylim(0,1.0)

    plt.title('Max depth = '+str(md)+', AdaBoost')
    
    if i == 2:
        plt.legend();
    
    plt.tight_layout()

plt.savefig('AdaB_performance.png')

### Revisión de aprendizaje

Mirando a esta figura, ¿recomendarias usar AdaBoost con un aprendiz de base con max_depth = 6, y 30 iteraciones, o con max_depth = 10 y 10 iteraciones?

<br>

<details>
<summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
<p>
    
```
Las notas de r2 y la correlación entre los valores verdaderos y previstos ambas son más altas para el caso de max_depth = 10 y 10 iteraciones, así que esa es la mejor opción. 
```
    
</p>
</details>

Tenemos una media-respuesta en el tercer panel de la figura, pero también se puede probar seguir con Boosting -- o, si agregar más etapas ayudará.

In [ ]:
# ¿Seguimos con boosting? (max_depth = 10)

n_estimators = 60

model= AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=10),
                  n_estimators=n_estimators)

X_train, X_test, y_train, y_test = \
        train_test_split(sel_features,sel_target.values.ravel(), test_size=.3, random_state=42)

model.fit(X_train, y_train)

plt.plot(range(n_estimators), [metrics.r2_score(y_test,list(model.staged_predict(X_test))[i]) for i in range(n_estimators)], label = 'r2')

plt.plot(range(n_estimators), [stats.spearmanr(y_test,list(model.staged_predict(X_test))[i])[0] for i in range(n_estimators)], label = 'Spearman r')

plt.xlabel('Iteration')

plt.title('Base estimator, max depth = 10')

plt.legend();


### Conclusión: combinando aprendices que son demasiado débiles no ayuda.

### ¿Sería lo mismo para algoritmos de Árboles 'Boosted' con Gradiente?

¡Sabemos cómo averiguarlo!

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

Los parámetros dependen de la implementación particular.

En la formulación de sklearn, los parámetros de cada árbol son casi lo mismo a los que teníamos para los Bosques Aleatorios. Además tenemos el parámetro "learning_rate" (velocidad de aprendizaje), lo cual decide cuanto cada árbol contribuye al estimador final, y los parámetros de "subsample", los cuales nos dejan usar menos que 100% de las muestras.

Podemos ver como funciona con un aprendiz débil en nuestro conjunto del ejemplo básico.

In [ ]:
# Crear el conjunto de datos
plt.figure(figsize=(15,12))

rng = np.random.RandomState(1)
X = np.linspace(0, 4, 100)[:, np.newaxis]
y = np.sin(X).ravel() + np.sin(6 * X).ravel() + rng.normal(0, 0.1, X.shape[0])

weakl = DecisionTreeRegressor(max_depth=3)

# Encajamos el modelo de regresión
regr_1 = weakl
""
regr_2 = GradientBoostingRegressor(max_depth=3,
                          n_estimators=2, random_state=rng)

regr_3 = GradientBoostingRegressor(max_depth=3,
                          n_estimators=3, random_state=rng)

regr_4 = GradientBoostingRegressor(max_depth=3,
                          n_estimators=4, random_state=rng)

regr_10 = GradientBoostingRegressor(max_depth=3,
                          n_estimators=10, random_state=rng)

regr_100 = GradientBoostingRegressor(max_depth=3,
                          n_estimators=100, random_state=rng)


regr_1.fit(X, y)
regr_2.fit(X, y)
regr_3.fit(X, y)
regr_4.fit(X, y)
regr_10.fit(X, y)
regr_100.fit(X, y)

# Predicción 
y_1 = regr_1.predict(X)
y_2 = regr_2.predict(X)
y_3 = regr_3.predict(X)
y_4 = regr_4.predict(X)
y_10 = regr_10.predict(X)
y_100 = regr_100.predict(X)

for yp in [y_1,y_2,y_3,y_4,y_10, y_100]:
    print('R2 score: ', np.round(metrics.r2_score(yp,y),3))

# Marcar los resultados.

plt.scatter(X, y, c="k", s=10,label="training samples")
plt.plot(X, y_1, "-g", label="n_estimators=1", linewidth=1)
#plt.plot(X, y_2, "--r", label="n_estimators=2", linewidth=1)
plt.plot(X, y_3, "-.b", label="n_estimators=3", linewidth=1)
#plt.plot(X, y_4, ":m", label="n_estimators=4", linewidth=1)
plt.plot(X, y_10, "-k", label="n_estimators=10", linewidth=1)
plt.plot(X, y_100, "-c", label="n_estimators=100", linewidth=1)
plt.xlabel("data")
plt.ylabel("target")
plt.ylim(-2.5,2.5)
plt.title("Gradient Boosting Regression, max depth = 3", fontsize = 14)
plt.legend(fontsize=14, loc = 'upper right');
#plt.tight_layout()
#plt.savefig("GradBoost_3.png")

### Revisión de aprendizaje
    
¡Hay notas de r2 para unos GBTs (árboles 'boosted' con gradiente) que son negativos! ¿No deben las notas de r2 siempre ser positivas?

<br>

<details>
<summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
<p>
    
```
No, ese requisito solo es para el conjunto de entrenamiento. Una nota negativa de r2 para el conjunto de prueba (o de validación) solo te dice que el desempeño del modélo es peor que una predicción constante equivalente al medio de la muestra. Seria un modelo horrible, pero no es necesariamente algo mal en nuestro código ( ;) ).


    
</p>
</details>
</br>

In [ ]:
plt.figure(figsize=(12,4))

n_estimators = 30

for i, md in enumerate([3,6,10]):
    
    model = GradientBoostingRegressor(max_depth=md,
                  n_estimators=n_estimators)

    model.fit(X_train,y_train)
    
    plt.subplot(1,3,i+1)

    plt.plot(range(n_estimators), [metrics.r2_score(y_test,list(model.staged_predict(X_test))[i]) for i in range(n_estimators)], label = 'r2 score', c = 'steelblue')

    plt.plot(range(n_estimators), [stats.spearmanr(y_test,list(model.staged_predict(X_test))[i])[0] for i in range(n_estimators)], label = 'Spearman r', c = 'fuchsia')

    plt.xlabel('Iteration')

    plt.ylim(0,1.0)

    plt.title('Max depth = '+str(md)+', GBR')
    
    if i == 2:
        plt.legend();
    
    plt.tight_layout()

plt.savefig('GBR_performance.png')

### Por causa del diferente proceso de boosting, los modelos de GBT funcionan bien hasta con aprendices de base débiles. 


En el próximo cuaderno, compararemos AdaBoost y muchos modelos de GBT para el problema de corrimientos al rojo fotométricos.